In [1]:
import copy
import itertools
import json
import os
import random
from typing import Any, Dict, List
import yaml

from jinja2 import Template, StrictUndefined
from jinja2.exceptions import TemplateError

from openai import AsyncOpenAI
from pydantic import BaseModel, Extra, Field

# pydantic-ai
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel, OpenAIResponsesModelSettings
from pydantic_ai.providers.openai import OpenAIProvider

from src.models import (
    CharacterSpecification,
    CharacterState,
    Character,
    EmotionalState,
    SocialRelationState,
    create_dynamic_enum
)
from config import settings

In [2]:
# Initialize OpenAI provider
print(settings.llm_model)

# Pydantic Model
provider = OpenAIProvider(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key
)
model = OpenAIModel(
    model_name=settings.llm_model,
    provider=provider
)

# OpenAI Client
aopenai_client = AsyncOpenAI(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key
)

gpt-4.1-nano


# 1. Prepare Scene & Character Data

In [3]:
## Scene Data
scene_uid = "2f2462c1-bdef-4089-b206-c47decd841f3"

with open(f"assets/simschat/scene/{scene_uid}.json", "r") as f:
    scene_dict = json.load(f)

print(json.dumps(scene_dict,indent=4))

{
    "uid": "2f2462c1-bdef-4089-b206-c47decd841f3",
    "stage": 0,
    "scene": {
        "revised_trope": "Inside the bustling grandeur of the metropolitan fashion gala, Zephyr Orion, a 28-year-old jocular astronaut with a penchant for playful storytelling, encounters Vivienne LaRoux. Vivienne, also 28, exudes sophistication and an assertive demeanor as a renowned style influencer. Initially, she greets Zephyr's lighthearted banter with icy indifference, her mean streak surfacing sporadically. However, Zephyr's infectious humor gradually softens her edges, revealing a subtly receptive side. Their verbal dance, rich with lively exchanges, challenges both to reconsider their outlooks, Zephyr embracing Vivienne's world of high fashion while she discovers a brighter perspective in his social magnetism.",
        "utilized_reference_tropes": [
            "Defrosting Ice Queen",
            "SavvyGuyEnergeticGirl"
        ],
        "scene": {
            "location": "The Metropolitan Fa

In [4]:
## Character Data
character_uid ="35f0c56f-263d-42df-846c-e1833d8ca0ab"
with open(f"assets/simschat/characters/{character_uid}.json", "r") as f:
    character_spec_dict = json.load(f)
    
character_spec = CharacterSpecification(**character_spec_dict)
# print(character_spec.model_dump_json(indent=2))

In [5]:
## Character State
neutral_emotional_state = EmotionalState(
    joy="na",
    trust="na",
    fear="na",
    surprise="na",
    sadness="na",
    disgust="na",
    anger="na",
    anticipation="na",
)

character_state_dict = scene_dict["character_states"]["character1"]

## Override previous emotion values with new spec
del character_state_dict["sentiment"]
del character_state_dict['emotion'] # remove emotion (replaced with plutchik)
character_state_dict["emotion"] = neutral_emotional_state.model_dump()
for x in character_state_dict['social_relations']:
    del x['emotion']
    x['emotion'] = neutral_emotional_state.model_dump()


character_state = CharacterState.model_validate(character_state_dict)

In [6]:
character = Character(
    uid=character_uid,
    spec=character_spec,
    state=character_state,
)

# 2. Initialize Prediction fn

In [7]:
with open('prompts/action_predictor.yaml', 'r') as file:
    action_predictor_prompt = yaml.load(file, Loader=yaml.FullLoader)

system_template = Template(
    action_predictor_prompt['system'],
    undefined=StrictUndefined
)
user_template = Template(
    action_predictor_prompt['user'],
    undefined=StrictUndefined
)

with open("assets/sims_interactions.json", "r") as f:
    all_actions = json.load(f)
n = 10
actions = {k: random.sample(v, min(len(v), n)) for k,v in all_actions.items()}
print(actions.keys())

system_contents = {
    "actions": actions
}
system_message = system_template.render(**system_contents)

dict_keys(['Friendly', 'Ask', 'Romantic', 'Mean', 'Neutral'])


In [8]:
## Make enum of actions
possible_actions_model = create_dynamic_enum(
    "PossibleCharacterActions",
    itertools.chain(*[
        [f"{k}-{a['action']}" for a in v]
        for k,v in actions.items()
    ])
)

In [9]:
# Action Prediction Tool Function
class CharacterAction(BaseModel):
    # character: str
    think: str
    action_type: possible_actions_model
    description: str
    dialogue: str
    targets: List[str]
    class Config:
        extra = Extra.forbid

class ActionPredictionResult(BaseModel):
    action: CharacterAction
    updated_state: CharacterState
    class Config:
        extra = Extra.forbid

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_70677/3145226243.py:10: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid
/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_70677/3145226243.py:16: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid


In [10]:
async def predict_action(
    character: Character,
    scene_context: str
)->ActionPredictionResult:
    user_message = user_template.render(
        character=character.model_dump_json(),
        scene_context=scene_context
    )
    print(user_message)
    result = await aopenai_client.beta.chat.completions.parse(
        model=settings.llm_model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message},
        ],
        response_format=ActionPredictionResult,
    )
    return result.choices[0].message.parsed

In [11]:
scene_context="Inside the bustling grandeur of the metropolitan fashion gala, Zephyr Orion, a 28-year-old jocular astronaut with a penchant for playful storytelling, encounters Vivienne LaRoux. Vivienne, also 28, exudes sophistication and an assertive demeanor as a renowned style influencer. Initially, she greets Zephyr's lighthearted banter with icy indifference, her mean streak surfacing sporadically. However, Zephyr's infectious humor gradually softens her edges, revealing a subtly receptive side. Their verbal dance, rich with lively exchanges, challenges both to reconsider their outlooks, Zephyr embracing Vivienne's world of high fashion while she discovers a brighter perspective in his social magnetism."
predicted_action = await predict_action(
    character=character,
    scene_context=scene_context
)

Information:
[Character]
{"uid":"35f0c56f-263d-42df-846c-e1833d8ca0ab","spec":{"name":"Zephyr Orion","gender":"male","age":28,"dialogue_tone":"playful, jovial, and engaging, with a witty humor and warmth that makes everyone feel at ease. Known for storytelling with captivating tales of space adventures.","career":"Astronaut","personality_traits":[{"trait":"Goofball","description":"Enjoys joking and making others laugh, bringing a playful spirit to social situations."},{"trait":"Materialistic","description":"Loves acquiring new possessions and often leans towards bragging about them."},{"trait":"Outgoing","description":"Flourishes in social situations and enjoys being around people."},{"trait":"Gloomy","description":"Grows sad when left alone for too long."},{"trait":"Ambitious","description":"Continuously strives to reach new milestones in his career."}],"hobbies":["Fitness","Cooking","Painting"],"living_conditions":["Resides in a modern city apartment filled with space memorabilia and

In [12]:
print(predicted_action.model_dump_json(indent=2))

{
  "action": {
    "think": "Zephyr should continue to engage Vivienne with his playful charisma, attempting to warm her up further and break through her icy exterior. He aims to entertain her and perhaps share a witty story to create a more relaxed and inviting atmosphere.",
    "action_type": "Friendly-Gestures/Greetings",
    "description": "Zephyr smiles warmly and gestures with an open hand, making a playful comment about the fashion in the room as a way to continue the lighthearted interaction.",
    "dialogue": "You know, with all these stunning outfits, I almost feel like I'm floating in a space parade—minus the zero gravity, thankfully.",
    "targets": [
      "00d66087-9b3b-46da-bd74-bf45cbe81d3c"
    ]
  },
  "updated_state": {
    "emotion": {
      "joy": "low",
      "trust": "low",
      "fear": "na",
      "surprise": "na",
      "sadness": "na",
      "disgust": "na",
      "anger": "na",
      "anticipation": "medium"
    },
    "social_relations": [
      {
       

# 3. CharacterActionPredictor Class

In [25]:
neutral_emotional_state = EmotionalState(
    joy="na",
    trust="na",
    fear="na",
    surprise="na",
    sadness="na",
    disgust="na",
    anger="na",
    anticipation="na",
)

class CharacterStore:
    def __init__(self, character_dir: str, uids: List[str]):
        self.characters = {}
        
        for uid in uids:
            # Spec
            with open(os.path.join(character_dir, f"{character_uid}.json"), "r") as f:
                spec_dict = json.load(f)
            spec = CharacterSpecification(**spec_dict)
            
            # Emotion
            emotional_state = copy.deepcopy(neutral_emotional_state)
            
            # Social Relations
            social_relations = []
            for target_uid in uids:
                if uid==target_uid:
                    continue
                relation = SocialRelationState(
                    character_uid=target_uid,
                    emotion = copy.deepcopy(neutral_emotional_state),
                    knowledge = []
                )
                social_relations.append(relation)
            state = CharacterState(
                emotion=emotional_state,
                social_relations=social_relations
            )
            character = Character(
                uid=uid,
                spec=spec,
                state=state
            )
            self.characters[uid]=character
            
    
    def get(self, uid: str) -> Character:
        return self.characters[uid]
 

character_dir = "assets/simschat/characters"
character_uids = [
    "35f0c56f-263d-42df-846c-e1833d8ca0ab",
    "00d66087-9b3b-46da-bd74-bf45cbe81d3c"
]
CHARACTER_STORE = CharacterStore(
    character_dir=character_dir,
    uids=character_uids
)

In [18]:
class CharcterActionPredictor:
    def __init__(
        self,
        prompt: Dict[str, str],
        actions: Dict[str, str],
        character_store: CharacterStore
    ):
        # Load Prompt
        system_template = Template(
            prompt['system'],
            undefined=StrictUndefined
        )
        self.system_message = system_template.render(actions=actions)
        
        self.user_template = Template(
            prompt['user'],
            undefined=StrictUndefined
        )
        
        # character store
        self.character_store=character_store
    
    def _get_character(self, uid: str) -> Character:
        return self.character_store.get(uid)
    
    async def predict_action(
        self,
        character: Character,
        scene_context: str
    )->ActionPredictionResult:
        user_message = self.user_template.render(
            character=character.model_dump_json(),
            scene_context=scene_context
        )
        print(user_message)
        result = await aopenai_client.beta.chat.completions.parse(
            model=settings.llm_model,
            messages=[
                {"role": "system", "content": self.system_message},
                {"role": "user", "content": user_message},
            ],
            response_format=ActionPredictionResult,
        )
        return result.choices[0].message.parsed
    
    def _update_character_state(self, character: Character, updated_state: CharacterState):
        character.state=updated_state
    
    async def forward(self, uid: str, scene_context: str):
        character = self._get_character(uid)
        
        # Predict Character Action
        result = await self.predict_action(
            character=character,
            scene_context=scene_context
        )
        
        # Update Character State
        self._update_character_state(
            character=character,
            updated_state=result.updated_state
        )
        return {
            "action": result.action.description,
            "targets": result.action.targets
        }

In [20]:
predictor = CharcterActionPredictor(
    prompt=action_predictor_prompt,
    actions=actions,
    character_store=CHARACTER_STORE
)

In [21]:
# Before Prediction
print(character.state)

emotion=EmotionalState(joy=<EmotionPolarity.na: 'na'>, trust=<EmotionPolarity.na: 'na'>, fear=<EmotionPolarity.na: 'na'>, surprise=<EmotionPolarity.na: 'na'>, sadness=<EmotionPolarity.na: 'na'>, disgust=<EmotionPolarity.na: 'na'>, anger=<EmotionPolarity.na: 'na'>, anticipation=<EmotionPolarity.na: 'na'>) social_relations=[SocialRelationState(character_uid='00d66087-9b3b-46da-bd74-bf45cbe81d3c', emotion=EmotionalState(joy=<EmotionPolarity.na: 'na'>, trust=<EmotionPolarity.na: 'na'>, fear=<EmotionPolarity.na: 'na'>, surprise=<EmotionPolarity.na: 'na'>, sadness=<EmotionPolarity.na: 'na'>, disgust=<EmotionPolarity.na: 'na'>, anger=<EmotionPolarity.na: 'na'>, anticipation=<EmotionPolarity.na: 'na'>), knowledge=['Vivienne LaRoux is a renowned style influencer with an assertive demeanor.', 'Vivienne initially greets Zephyr with icy indifference during their interaction at the gala.', "Despite Vivienne's dismissive nature, Zephyr sees potential for a deeper connection."])]


In [22]:
predicted_action = await predictor.forward(
    uid=character_uid,
    scene_context=scene_context
)

Information:
[Character]
{"uid":"35f0c56f-263d-42df-846c-e1833d8ca0ab","spec":{"name":"Zephyr Orion","gender":"male","age":28,"dialogue_tone":"playful, jovial, and engaging, with a witty humor and warmth that makes everyone feel at ease. Known for storytelling with captivating tales of space adventures.","career":"Astronaut","personality_traits":[{"trait":"Goofball","description":"Enjoys joking and making others laugh, bringing a playful spirit to social situations."},{"trait":"Materialistic","description":"Loves acquiring new possessions and often leans towards bragging about them."},{"trait":"Outgoing","description":"Flourishes in social situations and enjoys being around people."},{"trait":"Gloomy","description":"Grows sad when left alone for too long."},{"trait":"Ambitious","description":"Continuously strives to reach new milestones in his career."}],"hobbies":["Fitness","Cooking","Painting"],"living_conditions":["Resides in a modern city apartment filled with space memorabilia and

In [23]:
# After Update
print(character.state)

emotion=EmotionalState(joy=<EmotionPolarity.na: 'na'>, trust=<EmotionPolarity.na: 'na'>, fear=<EmotionPolarity.na: 'na'>, surprise=<EmotionPolarity.na: 'na'>, sadness=<EmotionPolarity.na: 'na'>, disgust=<EmotionPolarity.na: 'na'>, anger=<EmotionPolarity.na: 'na'>, anticipation=<EmotionPolarity.na: 'na'>) social_relations=[SocialRelationState(character_uid='00d66087-9b3b-46da-bd74-bf45cbe81d3c', emotion=EmotionalState(joy=<EmotionPolarity.na: 'na'>, trust=<EmotionPolarity.na: 'na'>, fear=<EmotionPolarity.na: 'na'>, surprise=<EmotionPolarity.na: 'na'>, sadness=<EmotionPolarity.na: 'na'>, disgust=<EmotionPolarity.na: 'na'>, anger=<EmotionPolarity.na: 'na'>, anticipation=<EmotionPolarity.na: 'na'>), knowledge=['Vivienne LaRoux is a renowned style influencer with an assertive demeanor.', 'Vivienne initially greets Zephyr with icy indifference during their interaction at the gala.', "Despite Vivienne's dismissive nature, Zephyr sees potential for a deeper connection."])]


In [24]:
print(json.dumps(predicted_action, indent=2))

{
  "action": "Zephyr offers a charming and playful greeting, perhaps using a witty compliment about her style or making a humorous remark about the gala, aiming to forge a more relaxed and jovial connection.",
  "targets": [
    "00d66087-9b3b-46da-bd74-bf45cbe81d3c"
  ]
}
